# Multimodal RAG v1

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

C:\Users\Anastasia\sber-reports-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# Функция извлечения элементов из pdf-файла
def extract_pdf_elements(path, fname):
    """
    Функция для извлечения различных элементов из PDF-файла, таких как изображения, таблицы,
    и текста. Также осуществляется разбиение текста на части (чанки) для дальнейшей обработки.

    Аргументы:
    path: Строка, содержащая путь к директории, в которую будут сохранены извлеченные изображения.
    fname: Строка, содержащая имя PDF-файла, который необходимо обработать.

    Возвращает:
    Список объектов типа `unstructured.documents.elements`, представляющих извлеченные из PDF элементы.
    """
    return partition_pdf(
        filename=path + fname,          # Путь к файлу, который нужно обработать
        extract_images_in_pdf=True,     # Указание на то, что из PDF нужно извлечь изображения
        infer_table_structure=True,     # Автоматическое определение структуры таблиц в документе
        strategy="hi_res",              # Стратегия разбиения текста на части
        max_characters=1024,            # Максимальное количество символов в одном чанке текста
        new_after_n_chars=1024,         # Число символов, после которого начинается новый чанк текста
        combine_text_under_n_chars=512, # Минимальное количество символов, при котором чанки объединяются
        image_output_dir_path=path,     # Путь, куда будут сохраняться извлеченные изображения
    )

# Указываем путь до директории, где находится PDF-файл
fpath = "../data/raw/"
# Указываем имя PDF-файла, который нужно обработать
fname = "Сбер 2023-1-20.pdf"

# Извлекаем элементы из PDF-файла с помощью функции extract_pdf_elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

In [27]:
raw_pdf_elements

In [70]:
raw_pdf_elements[-5].text

'Стоимость риска (COR)'

In [62]:
# Функция категоризации элементов
def categorize_elements(raw_pdf_elements):
    """
    Функция для категоризации извлеченных элементов из PDF-файла.
    Элементы делятся на текстовые элементы и таблицы.

    Аргументы:
    raw_pdf_elements: Список объектов типа `unstructured.documents.elements`,
                      представляющих извлеченные из PDF элементы.

    Возвращает:
    Два списка: texts (текстовые элементы) и tables (таблицы).
    """
    tables = []  # Список для хранения элементов типа "таблица"
    texts = []   # Список для хранения текстовых элементов
    for element in raw_pdf_elements:
        # Проверка типа элемента. Если элемент является таблицей, добавляем его в список таблиц
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        # Если элемент является композитным текстовым элементом, добавляем его в список текстов
        elif ("unstructured.documents.elements.Text" in str(type(element))
        or "unstructured.documents.elements.Title" in str(type(element))
        or "unstructured.documents.elements.NarrativeText" in str(type(element))
        or "unstructured.documents.elements.ListItem" in str(type(element))): 
            texts.append(str(element))
        elif "unstructured.documents.elements.Header" in str(type(element)):
            texts.append("## " + str(element))
    return texts, tables  # Возвращаем списки с текстами и таблицами

In [63]:
# Категоризируем извлеченные элементы на текстовые и табличные с помощью функции categorize_elements
texts, tables = categorize_elements(raw_pdf_elements)

# Создаем объект CharacterTextSplitter для разбиения текста на части (чанки)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, # Максимальный размер чанка в символах
    chunk_overlap=0 # Количество перекрывающихся символов между чанками
)

# Объединяем все текстовые элементы в одну строку
joined_texts = " ".join(texts)

In [64]:
joined_texts

'3 В Отчете освещаются события за 2023 календарный год, за исключением случаев, прямо указанных по тексту. 3 Перейти ~~ OTueT MeHeDDKMECHTa on a —— 5 ## © CBEP ## Обращение Председателя Наблюдательного совета Отчет менеджмента ## (fe Обращение Председателя Наблюдательного совета Уважаемые клиенты и акционеры! 2023 год стал для российской экономики достаточно успешным благодаря эффективной структурной пере- стройке и адаптации бизнеса к новым реалиям. Деловая активность в ключевых отраслях экономики восстанав- ливалась, увеличивались инвестиции и потребление домохозяйств. В результате ВВП вырос на 3,6%. Сбер продолжил активно развивать направление, связанное с использованием искусственного интел- лекта. В 2023 году он представил рынку мультимо- дальную нейросеть GigaChat, а также обновленную версию Kandinsky 3.0. Тексты по финансовой грамотно- сти от GigaChat и картинки, сгенерированные под них Kandinsky 3.0, представлены на стенде Министерства финансов на выставке «Россия» на ВДНХ. Фин

In [71]:
# Разбиваем объединенный текст на чанки, используя созданный CharacterTextSplitter
texts_4k_token = text_splitter.split_text(joined_texts)

In [81]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# Функция для суммаризации текста и таблиц
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Функция для создания суммаризации текста и таблиц с использованием модели GPT.

    Аргументы:
    texts: Список строк (тексты), которые нужно суммировать.
    tables: Список строк (таблицы), которые нужно суммировать.
    summarize_texts: Булев флаг, указывающий, нужно ли суммировать текстовые элементы.

    Возвращает:
    Два списка: text_summaries (суммаризации текстов) и table_summaries (суммаризации таблиц).
    """

    # Шаблон для запроса к модели. Задача ассистента - создать оптимизированное описание для поиска.
    prompt_text = """ Суммаризируй кратко следующий элемент {element}.
    Сохрани ключевую информацию. Ответ должен содержать 2 предложения."""

    # Создаем шаблон запроса на основе строки с шаблоном
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Создаем модель для генерации суммаризаций. Устанавливаем температуру 0 для детерминированных ответов.
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini")

    # Определяем цепочку обработки запросов: сначала шаблон запроса, затем модель, затем парсер выходных данных
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    text_summaries = []  # Список для хранения суммаризаций текстов
    table_summaries = []  # Список для хранения суммаризаций таблиц

    # Если есть текстовые элементы и требуется их суммирование
    if texts and summarize_texts:
        # Выполняем параллельное суммирование текстов
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 10})# Ваш код здесь})
    elif texts:
        # Если суммирование не требуется, просто передаем исходные тексты
        text_summaries = texts

    # Если есть таблицы, выполняем их суммирование
    if tables:
        # Выполняем параллельное суммирование таблиц
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 10})# Ваш код здесь})

    return text_summaries, table_summaries  # Возвращаем результаты суммаризации

# Вызываем функцию для суммаризации текстов и таблиц, указывая, что нужно суммировать тексты
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [83]:
import base64
import os

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI


# Функция кодирования изображения в формат base64
def encode_image(image_path):
    """
    Функция для кодирования изображения в формат base64.

    Аргументы:
    image_path: Строка, путь к изображению, которое нужно закодировать.

    Возвращает:
    Закодированное в формате base64 изображение в виде строки.
    """
    with open(image_path, "rb") as image_file:
        # Читаем файл изображения в бинарном режиме и кодируем в base64
        return base64.b64encode(image_file.read()).decode("utf-8")


# Функция для суммаризации изображения с использованием модели GPT
def image_summarize(img_base64, prompt):
    """
    Функция для получения суммаризации изображения с использованием GPT модели.

    Аргументы:
    img_base64: Строка, изображение закодированное в формате base64.
    prompt: Строка, запрос для модели GPT, содержащий инструкцию для суммаризации изображения.

    Возвращает:
    Суммаризация изображения, возвращенная моделью GPT.
    """
    # Создаем объект модели GPT с заданными параметрами
    chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=400)

    # Отправляем запрос к модели GPT
    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},  # Запрос для модели
                    {
                        "type": "image_url",  # Тип содержимого - изображение
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},  # Изображение в формате base64
                    },
                ]
            )
        ]
    )
    # Возвращаем содержимое ответа от модели
    return msg.content


def generate_img_summaries(path):
    """
    Функция для генерации суммаризаций изображений из указанной директории.

    Аргументы:
    path: Строка, путь к директории с изображениями формата .jpg.

    Возвращает:
    Два списка:
    - img_base64_list: Список закодированных изображений в формате base64.
    - image_summaries: Список суммаризаций для каждого изображения.
    """
    img_base64_list = []  # Список для хранения закодированных изображений
    image_summaries = []  # Список для хранения суммаризаций изображений

    # Запрос для модели GPT
    prompt = """Опиши ключевую информацию, которая представлена на изображении. Описание должно быть конкретным и точным, а также содержать 2-3 предложения. Обрати особое внимание на графики, диаграммы или визуальные элементы, которые можно проанализировать."""

    # Обрабатываем все файлы в указанной директории
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):  # Проверяем, что файл имеет расширение .jpg
            img_path = os.path.join(path, img_file)  # Полный путь к изображению
            base64_image = encode_image(img_path)  # Кодируем изображение в base64
            img_base64_list.append(base64_image)  # Добавляем закодированное изображение в список
            image_summaries.append(image_summarize(base64_image, prompt))  # Получаем суммаризацию изображения

    return img_base64_list, image_summaries  # Возвращаем результаты


# Вызываем функцию для генерации суммаризаций изображений
img_base64_list, image_summaries = generate_img_summaries("figures/")

In [85]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [86]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

C:\Users\Anastasia\sber-reports-rag\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anastasia\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [87]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
# from langchain_openai import OpenAIEmbeddings


# Функция создания многофакторного ритривера для базы данных
def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Функция для создания ретривера, который может извлекать данные из разных источников (тексты, таблицы, изображения).

    Аргументы:
    vectorstore: Векторное хранилище для хранения векторных представлений документов.
    text_summaries: Список суммаризаций текстовых элементов.
    texts: Список исходных текстов.
    table_summaries: Список суммаризаций таблиц.
    tables: Список исходных таблиц.
    image_summaries: Список суммаризаций изображений.
    images: Список изображений в формате base64.

    Возвращает:
    Созданный ретривер, который может извлекать данные из различных источников.
    """

    # Создаем хранилище для метаданных документов в памяти
    store = InMemoryStore()
    id_key = "doc_id"  # Ключ для идентификации документов в хранилище

    # Создаем многофакторный ритривер
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Функция добавления документов в ритривер
    def add_documents(retriever, doc_summaries, doc_contents):
        """
        Функция для добавления документов и их метаданных в ритривер.

        Аргументы:
        retriever: Ретривер, в который будут добавляться документы.
        doc_summaries: Список суммаризаций документов.
        doc_contents: Список исходных содержимых документов.
        """
        # Генерируем уникальные идентификаторы для каждого документа
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]

        # Создаем документы для векторного хранилища из суммаризаций
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]

        # Добавляем документы в векторное хранилище
        retriever.vectorstore.add_documents(summary_docs)

        # Добавляем метаданные документов в хранилище
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Добавляем суммаризации текстов и таблиц, если они присутствуют
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever  # Возвращаем созданный ритривер


# Создаем векторное хранилище для хранения векторных представлений документов
vectorstore = Chroma(
    collection_name="mm_rag_sber",  # Название коллекции
    embedding_function=embeddings  # Функция для создания векторных представлений
)

# Создаем ретривер, добавляя суммаризации текстов, таблиц и изображений
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

C:\Users\Anastasia\AppData\Local\Temp\ipykernel_8236\2737274567.py:78: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [88]:
import io
import re
import base64

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


def plt_img_base64(img_base64):
    """
    Функция для отображения изображения, закодированного в формате base64.

    Аргументы:
    img_base64: Закодированное в формате base64 изображение.
    """
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    display(HTML(image_html))


def looks_like_base64(sb):
    """
    Проверяет, выглядит ли строка как base64.

    Аргументы:
    sb: Строка для проверки.

    Возвращает:
    True, если строка выглядит как base64, иначе False.
    """
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Проверяет, является ли base64 данные изображением, проверяя сигнатуры данных.

    Аргументы:
    b64data: Строка base64, представляющая изображение.

    Возвращает:
    True, если данные начинаются с сигнатуры изображения, иначе False.
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Изменяет размер изображения, закодированного в формате base64.

    Аргументы:
    base64_string: Строка base64, представляющая изображение.
    size: Новый размер изображения.

    Возвращает:
    Закодированное в формате base64 изображение нового размера.
    """
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Изменение размера изображения с использованием алгоритма LANCZOS для улучшения качества
    resized_img = img.resize(size, Image.LANCZOS)

    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Разделяет документы на изображения и текстовые данные.

    Аргументы:
    docs: Список документов, содержащих изображения (в формате base64) и текст.

    Возвращает:
    Словарь с двумя списками: изображения и тексты.
    """
    b64_images = []
    texts = []
    for doc in docs:
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


# Функция формирования запроса для модели с учетом изображений и текста
def img_prompt_func(data_dict):
    """
    Формирует запрос к модели с учетом изображений и текста.

    Аргументы:
    data_dict: Словарь, содержащий тексты и изображения, а также вопрос пользователя.

    Возвращает:
    Список сообщений для отправки модели.
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Добавляем изображения в сообщения, если они присутствуют
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Формируем текстовое сообщение с вопросом пользователя и текстовыми данными
    text_message = {
        "type": "text",
        "text": (
            # Ваш код здесь
            f"Вопрос пользователя: {data_dict['question']}\n\n"
            "Текст и / или таблицы:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Создает RAG цепочку для работы с мультимодальными запросами, включая текст и изображения.

    Аргументы:
    retriever: Ритривер для получения данных.

    Возвращает:
    Цепочка для обработки запросов с учетом текста и изображений.
    """
    model = ChatOpenAI(temperature=0.2, model="gpt-4o-mini")

    # Определяем цепочку обработки запросов
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


# Создаем RAG цепочку с использованием ретривера
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [89]:
# Пример запроса
query = "Как сбер адаптировался к новым условиям?"
docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)

C:\Users\Anastasia\AppData\Local\Temp\ipykernel_8236\75163073.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)


In [91]:
print(chain_multimodal_rag.invoke(query)) 

Сбер адаптировался к новым условиям, внедрив несколько ключевых стратегий и инициатив:

1. **Цифровизация**: Сбер стал лидером в цифровом банкинге, предлагая лучшие банковские приложения и практики управления рисками.

2. **Интеграция технологий**: Создание интегрированной технологической экосистемы вокруг клиента, что позволяет объединять финансовые и нефинансовые сервисы.

3. **Человекоцентричность**: Сбер стремится стать помощником человека в управлении настоящим и будущим, включая использование искусственного интеллекта.

4. **Рост клиентской базы**: Увеличение числа активных клиентов как среди розничных, так и корпоративных пользователей.

Эти меры позволили Сберу укрепить свои позиции на рынке и адаптироваться к изменениям в экономической среде.
